### Exploring and Using Frontier Model APIs

##### Suggested Description

This project aims to connect with and experiment with advanced language models (“Frontier Models”) through their official APIs. After using various models via their chat interfaces and working with the OpenAI API in the first week, we will now expand our scope by interacting programmatically with multiple providers, such as Anthropic, Gemini, and others, provided we have their access credentials.

The goal is to send queries to different models, compare their responses, and explore the capabilities, differences, and behaviors of each API. Integration with additional providers is entirely optional, allowing each participant to customize their environment according to the tools they have access to.

In [29]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic as ant
import google.generativeai as genai

In [30]:
load_dotenv(override=True)

API_KEYS = {
    "OpenAI": ("OPENAI_API_KEY", 8),
    "Anthropic": ("ANTHROPIC_API_KEY", 7),
    "Google": ("GOOGLE_API_KEY", 2)
}

keys = {label: os.getenv(env) for label, (env, _) in API_KEYS.items()}

def check_key(label, prefix_len):
    key = keys[label]
    if key:
        print(f"{label} API Key exists and begins {key[:prefix_len]}")
    else:
        print(f"{label} API Key not set (optional)")

for label, (env_name, prefix_len) in API_KEYS.items():
    check_key(label, prefix_len)


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI


In [31]:
openai  = OpenAI(api_key=keys["OpenAI"])
anthropic = ant.Anthropic(api_key=keys["Anthropic"])
genai.configure(api_key=keys["Google"])

In [32]:
MAX_TOKENS = 1024
MODEL = "gpt-4.1-mini"
SYSTEM_PROMPT = "You are a very helpful assistant"
USER_PROMPT = [
    { "role": "user", "content": "How can I decide if a business problem is suitable for an LLM solution? Answer in Markdown" }
]

## The Internal Structure of the Message History

Originally, `gradio` expected to receive a function called:

`chat(message, history)`

This function had to receive `history` in a specific format, which we had to assign to the OpenAI format before making the call:

```json
[
{"role": "system", "content": "system message here"},
{"role": "user", "content": "first user prompt here"},
{"role": "assistant", "content": "the assistant's response"},
{"role": "user", "content": "the new user prompt"}
]
```
### Gradio has been updated! 🚀

Now it will pass `history` in OpenAI's **exact** format, perfect for sending directly to the API. So our work just got easier!

#### 📋 Action Plan:

We'll write a function `chat(message, history)` where:

* **`message`**: This is the current message to use.

* **`history`**: This is the previous conversation (already in OpenAI-compatible format).

We'll combine the **system message**, the **history**, and the **last message**, and then call OpenAI.


In [5]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [9]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history + [{"role": "user", "content": message}]

    print("history:", history)
    print("messages:", messages)
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat).launch()

## OK let's keep going!

Using a system message to add context, and to give an example answer.. this is "one shot prompting"

In [33]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [34]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat).launch()

In [14]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [17]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if re.search(r"\bbelts?\b", message, re.IGNORECASE):
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat).launch()

# Project - PPTX summarizer AI Assistant

In [73]:
from pptx import Presentation
import json

class PptxSummary:
    def __init__(self, name: str):
        self.name = name

    def summary(self):
        # Load an existing PowerPoint
        prs = Presentation(self.name)
        newtext = ""

        # print(prs)
        # Loop over all slides
        for i, slide in enumerate(prs.slides, start=1):
            # print(f"\n--- Slide {i} ---")
            newtext += f"\n\n--- Slide {i} ---"
            
            # Loop over shapes (text boxes, titles, placeholders, etc.)
            for shape in slide.shapes:
                if shape.has_text_frame:  # Only shapes that can contain text
                    for paragraph in shape.text_frame.paragraphs:
                        # Collect text from each run in the paragraph
                        text = "".join(run.text for run in paragraph.runs)
                        # print(text)
                        newtext+= "\n"
                        newtext += text
        # print(newtext)
        return newtext

        
system_message = "You are a helpful assistant for a company and you can summarize the given topic based on the given pptx name. "
system_message += "Give short, courteous answers, no more than 10 sentence. "
system_message += "Always be accurate. If the presentation .pptx file does not exist, say so. Respond in markdown."

def user_message(path):
    ppt = PptxSummary(path)
    summarization_message = ppt.summary()
    message = "You need to summarize the a pptx file."
    message += f"The context of that pptx file is here: {summarization_message}"
    message += "Give the concise information in small paragraphs. "
    return message

def pptx_summary(path):
    if os.path.exists(path):
        result = "The file does not exist"
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message(path)}
              ],
        )
        result = response.choices[0].message.content
        return result
    else:
        return "The file does not exist"

In [59]:
print(pptx_summary("data/conversational_chatbot/Miguel_Grau.pptx"))

The presentation titled "Miguel Grau" appears to focus on an individual named Miguel Grau. Unfortunately, the information provided is minimal. The second slide mentions a birth date of "30 de febrero de 1987," which is not a valid date, indicating a possible error. No further details or context are available from the slides provided. If you have more slides or information, I can help provide a more detailed summary.


In [60]:
summary_function = {
    "name": "pptx_summary",
    "description": "Get the summary for the given pptx file, you need to call this function, if user asks for a pptx file, if it is outside of the pptx file, tell that you do not know.",
    "parameters": {
        "type": "object",
        "properties": {
            "path": {
                "type": "string",
                "description": "The name of the presentation",
            },
        },
        "required": ["path"],
        "additionalProperties": False
    }
}

In [61]:
tools = [{"type": "function", "function": summary_function}]

In [78]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(f"tool_call: {tool_call}")
    arguments = json.loads(tool_call.function.arguments)
    print(f"arguments: {arguments}")
    path = f"data/conversational_chatbot/{arguments.get('path')}"
    print(f"path: {path}")
    summary = pptx_summary(path)
    # print(f"price: {price}")
    response = {
        "role": "tool",
        "content": json.dumps({"path": path,"summary": summary}),
        "tool_call_id": tool_call.id
    }
    print(f"response: {response}")
    return response

In [85]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(f"response 00: {response}")
    print(f"response 00xx: {response.choices[0].message.content}")

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(f"message: {message}")
        response = handle_tool_call(message)
        print(f"response 01: {response}")
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        print(f"response 02: {response}")
        print(f"response 03: {response.choices[0].message.content}")
            
    return response.choices[0].message.content

In [86]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


response 00: ChatCompletion(id='chatcmpl-CllhT7BhlUGstfjPHFaxs3u3NPuHx', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_IJ8AxNQ6AjYQJricVSRBolRF', function=Function(arguments='{"path":"Miguel_Grau.pptx"}', name='pptx_summary'), type='function')]))], created=1765500811, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_7abc656409', usage=CompletionUsage(completion_tokens=22, prompt_tokens=153, total_tokens=175, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
response 00xx: None
message: ChatCompletionMessage(content=None, refusal=None, role='assistant', annot